# 5. Deep Learning Computation

In [1]:
from mxnet import np, npx
from mxnet.gluon import nn

npx.set_np()

In [2]:
x = np.random.uniform(size = (2,20))

In [5]:
net = nn.Sequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize()
net(x)

array([[ 0.06240272, -0.03268594,  0.02582652,  0.02254181, -0.03728798,
        -0.04253786,  0.00540613, -0.01364184, -0.09915452, -0.02272737],
       [ 0.02816678, -0.03341203,  0.03565667,  0.02506383, -0.04136416,
        -0.04941843,  0.01738529,  0.01081963, -0.09932576, -0.01176296]])

### 5.1.1 A Custom Block

Coding up a block from scratch

Our MLP class below inherits the Block class. While we rely on some predefined methods in the parent class, we need to supply our own `__init__` and forward functions to uniquely define the behavior of our model

In [7]:
from mxnet.gluon import nn

In [8]:
class MLP(nn.Block):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self, **kwargs):
        # Call the constructor of the MLP parent class BLock to perform the 
        # necessary initialization. In this way, other function parameters can
        # also be specified when constructing an instance, such as the model parameters,
        # params, described in the following sections
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Dense(256, activation='relu')
        self.output = nn.Dense(10)
        
    # Define the forward computation of the model, that is, how to return the
    # required model output based on the input x
    def forward(self, x):
        return self.output(self.hidden(x))